## Getting IoT Data

### Load Required Packages

In [2]:
!pip install python-dotenv

You should consider upgrading via the 'D:\soch1\Projects\IoT_course\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
import requests
import csv
from datetime import datetime
import json
import os
from dotenv import load_dotenv


In [4]:
def login(url):
    load_dotenv()
    # Define the username and password
    post_data = {
        'username': os.environ.get('VIZHUB_LOGIN'), 
        'password': os.environ.get('VIZHUB_PW') 
    }

    # Define the headers
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }

    # Send the POST request
    # response = requests.post(url, data=data)
    response = requests.post(url, headers=headers, data=json.dumps(post_data))

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Failed to fetch data from ThingsBoard API. Status code:", response.status_code)
        return None


In [5]:
def fetch_sensor_data(device_id, sensor_key, start_ts, end_ts, access_token):
    url = f'http://locationai.tech:8080/api/plugins/telemetry/DEVICE/{device_id}/values/timeseries?keys={sensor_key}'
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'X-Authorization': f'Bearer {access_token}'
    }
    params = {
        'startTs': start_ts,
        'endTs': end_ts
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Failed to fetch data from ThingsBoard API. Status code:", response.status_code)
        return None

In [6]:
def timestamp_to_datetime(timestamp):
    return datetime.utcfromtimestamp(timestamp / 1000).strftime('%Y-%m-%d %H:%M:%S')

## Getting Data from IoT Platform

In [7]:
url = f'http://locationai.tech:8080/api/auth/login'
response_data = login(url)


In [8]:
access_token = response_data['token']

In [9]:
device_id = '58366820-ae63-11ee-a39c-0f270afb2199' #STEM2

In [10]:
sensor_key = "CO2"
start_ts = 1700000000000
end_ts = 1710000000000

In [11]:
data = fetch_sensor_data(device_id, sensor_key, start_ts, end_ts, access_token)

Save to CSV file

In [12]:
if data and sensor_key in data:
        timestamps = [entry['ts'] for entry in data[sensor_key]]
        values = [entry['value'] for entry in data[sensor_key]]

        # Convert timestamps to datetime strings
        timestamps = [timestamp_to_datetime(ts) for ts in timestamps]

        with open(f'csv_data/thingsboard_{sensor_key.lower()}_data.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Timestamp', sensor_key])
            writer.writerows(zip(timestamps, values))

        print(f"Data written to thingsboard_{sensor_key.lower()}_data.csv")
else:
        print(f"No data available for the specified sensor ({sensor_key}) or time range.")

Data written to thingsboard_co2_data.csv


In [13]:
#data

## Organizing the Data

### Converting Data to Dataframe

In [14]:
import pandas as pd

In [15]:
# Convert JSON to DataFrame
df_CO2 = pd.json_normalize(data['CO2'])


In [16]:
df_CO2.head()

,ts,value
0,1708458900499,416
1,1708458878642,408
2,1708458856767,413
3,1708458834993,411
4,1708458813173,409


In [17]:
# Convert the 'ts' column to datetime
df_CO2['ts'] = pd.to_datetime(df_CO2['ts'], unit='ms')

In [18]:
df_CO2.head()

,ts,value
0,2024-02-20 19:55:00.499,416
1,2024-02-20 19:54:38.642,408
2,2024-02-20 19:54:16.767,413
3,2024-02-20 19:53:54.993,411
4,2024-02-20 19:53:33.173,409


In [19]:
df_CO2.rename(columns={"ts": "Lastupdatetime"}, inplace=True)
df_CO2.rename(columns={"value": "CO2"}, inplace=True)

In [20]:
df_CO2.head()

,Lastupdatetime,CO2
0,2024-02-20 19:55:00.499,416
1,2024-02-20 19:54:38.642,408
2,2024-02-20 19:54:16.767,413
3,2024-02-20 19:53:54.993,411
4,2024-02-20 19:53:33.173,409


### Getting Temperature Data

In [21]:
from datetime import datetime

def convert_to_timestamp(date_string):
    try:
        dt_object = datetime.strptime(date_string, "%Y-%m-%d %H:%M")
        timestamp = dt_object.timestamp() * 1000  # Convert to milliseconds
        return int(timestamp)
    except ValueError:
        return "Wrong Date/Time Format"


In [22]:
sensor_key = "Temperature"
start_ts = convert_to_timestamp("2024-02-20 00:01")
end_ts = convert_to_timestamp("2024-02-20 23:00")

In [23]:
temperature_data = fetch_sensor_data(device_id, sensor_key, start_ts, end_ts, access_token)

In [24]:
# Convert JSON to DataFrame
df_temp = pd.json_normalize(temperature_data['Temperature'])

In [25]:
df_temp.head()

,ts,value
0,1708458900499,18
1,1708458878642,18
2,1708458856767,18
3,1708458834993,18
4,1708458813173,18


In [26]:
# Convert 'ts' to datetime format to match 'Lastupdatetime'
df_temp['ts'] = pd.to_datetime(df_temp['ts'], unit='ms')

# Rename 'ts' to 'Lastupdatetime' in df_temp
df_temp.rename(columns={'ts': 'Lastupdatetime'}, inplace=True)

In [27]:
df_temp.rename(columns={"value": "Temperature"}, inplace=True)

In [28]:
df_temp.head()

,Lastupdatetime,Temperature
0,2024-02-20 19:55:00.499,18
1,2024-02-20 19:54:38.642,18
2,2024-02-20 19:54:16.767,18
3,2024-02-20 19:53:54.993,18
4,2024-02-20 19:53:33.173,18


In [29]:
# Merge the two dataframes on 'Lastupdatetime'
df_merged = pd.merge(df_CO2, df_temp, on='Lastupdatetime')

In [30]:
df_merged.head()

,Lastupdatetime,CO2,Temperature
0,2024-02-20 19:55:00.499,416,18
1,2024-02-20 19:54:38.642,408,18
2,2024-02-20 19:54:16.767,413,18
3,2024-02-20 19:53:54.993,411,18
4,2024-02-20 19:53:33.173,409,18


### Time Series 
Let's visualize the CO / temperature in hourly basis.

In [31]:
aq= df_merged.rename(columns={'Lastupdatetime':'DateTime'})

In [32]:
aq['Weekday'] = aq['DateTime'].dt.day_name()
aq['Month']   = aq['DateTime'].dt.month_name()
aq['Hour']    = aq['DateTime'].dt.hour
aq['Date'] = aq['DateTime'].dt.date

aq.head()

,DateTime,CO2,Temperature,Weekday,Month,Hour,Date
0,2024-02-20 19:55:00.499,416,18,Tuesday,February,19,2024-02-20
1,2024-02-20 19:54:38.642,408,18,Tuesday,February,19,2024-02-20
2,2024-02-20 19:54:16.767,413,18,Tuesday,February,19,2024-02-20
3,2024-02-20 19:53:54.993,411,18,Tuesday,February,19,2024-02-20
4,2024-02-20 19:53:33.173,409,18,Tuesday,February,19,2024-02-20


In [33]:
aq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   DateTime     100 non-null    datetime64[ns]
 1   CO2          100 non-null    object        
 2   Temperature  100 non-null    object        
 3   Weekday      100 non-null    object        
 4   Month        100 non-null    object        
 5   Hour         100 non-null    int32         
 6   Date         100 non-null    object        
dtypes: datetime64[ns](1), int32(1), object(5)
memory usage: 5.2+ KB


In [35]:
aq['CO2'] = aq['CO2'].astype(int)
aq['Temperature'] = aq['Temperature'].astype(int)

In [36]:
aq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   DateTime     100 non-null    datetime64[ns]
 1   CO2          100 non-null    int32         
 2   Temperature  100 non-null    int32         
 3   Weekday      100 non-null    object        
 4   Month        100 non-null    object        
 5   Hour         100 non-null    int32         
 6   Date         100 non-null    object        
dtypes: datetime64[ns](1), int32(3), object(3)
memory usage: 4.4+ KB
